# 📊 Análise Exploratória de Dados - Cotações de Boi Gordo

Este notebook demonstra como fazer análise exploratória dos dados de cotações coletados.

## Objetivos:
- Carregar e visualizar dados históricos
- Analisar tendências temporais
- Comparar preços entre estados
- Identificar padrões sazonais
- Análise de volatilidade

## 1. Importações e Configurações

In [ ]:
# Importações básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import sys
import os

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Adicionar src ao path
sys.path.append('../src')

# Configurar matplotlib para português
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 2. Carregamento de Dados

In [ ]:
from data_science_loader import DataScience_Loader

# Inicializar loader
loader = DataScience_Loader("../output/parquet")

# Ver informações disponíveis
info = loader.get_data_info()
print("📊 Dados Disponíveis:")
print(f"Localização: {info['base_path']}")
print(f"Tabelas: {', '.join(info['available_tables'])}")
print()

for table, details in info['table_details'].items():
    print(f"🔹 {table}:")
    print(f"   Arquivos: {details['file_count']}")
    print(f"   Colunas: {', '.join(details['columns'])}")
    print()

In [ ]:
# Carregar dados dos últimos 6 meses
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%d')

print(f"📅 Carregando dados de {start_date} até {end_date}")

# Carregar dados por estado
ts_data = loader.load_time_series(
    start_date=start_date,
    end_date=end_date,
    table_type="indicadores_estados",
    states=None,  # Todos os estados
    indicators=None  # Todos os indicadores
)

print(f"✅ Dados carregados: {len(ts_data)} registros")
if not ts_data.empty:
    print(f"📊 Período: {ts_data.index.min()} até {ts_data.index.max()}")
    print(f"🗺️ Estados: {ts_data['state'].nunique()}")
    print(f"📈 Indicadores: {ts_data['indicator_name'].nunique()}")

## 3. Visão Geral dos Dados

In [ ]:
if not ts_data.empty:
    # Estatísticas básicas
    print("📊 ESTATÍSTICAS DESCRITIVAS")
    print("=" * 50)
    print(ts_data['price_brl'].describe())
    print()
    
    # Estados únicos
    print("🗺️ ESTADOS DISPONÍVEIS:")
    states = sorted(ts_data['state'].unique())
    for i, state in enumerate(states, 1):
        print(f"{i:2d}. {state}")
    
    print()
    
    # Indicadores únicos
    print("📈 INDICADORES DISPONÍVEIS:")
    indicators = sorted(ts_data['indicator_name'].unique())
    for i, indicator in enumerate(indicators, 1):
        print(f"{i:2d}. {indicator}")

## 4. Análise Temporal

In [ ]:
if not ts_data.empty:
    # Preço médio por data
    daily_avg = ts_data.groupby('date')['price_brl'].mean()
    
    # Gráfico de série temporal
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # Série temporal principal
    axes[0].plot(daily_avg.index, daily_avg.values, linewidth=2, color='darkblue')
    axes[0].set_title('📈 Evolução do Preço Médio do Boi Gordo', fontsize=16, fontweight='bold')
    axes[0].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Adicionar média móvel
    ma_7 = daily_avg.rolling(7).mean()
    ma_30 = daily_avg.rolling(30).mean()
    
    axes[0].plot(ma_7.index, ma_7.values, '--', alpha=0.7, label='Média 7 dias', color='orange')
    axes[0].plot(ma_30.index, ma_30.values, '--', alpha=0.7, label='Média 30 dias', color='red')
    axes[0].legend()
    
    # Volatilidade
    daily_vol = ts_data.groupby('date')['price_brl'].std()
    axes[1].plot(daily_vol.index, daily_vol.values, color='red', alpha=0.7)
    axes[1].set_title('📊 Volatilidade Diária dos Preços', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Desvio Padrão', fontsize=12)
    axes[1].set_xlabel('Data', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas
    print(f"💰 Preço médio no período: R$ {daily_avg.mean():.2f}")
    print(f"📈 Maior preço: R$ {daily_avg.max():.2f} em {daily_avg.idxmax().strftime('%d/%m/%Y')}")
    print(f"📉 Menor preço: R$ {daily_avg.min():.2f} em {daily_avg.idxmin().strftime('%d/%m/%Y')}")
    print(f"📊 Volatilidade média: ±R$ {daily_vol.mean():.2f}")

## 5. Análise por Estados

In [ ]:
if not ts_data.empty:
    # Preço médio por estado
    state_avg = ts_data.groupby('state')['price_brl'].agg(['mean', 'std', 'count']).sort_values('mean', ascending=False)
    
    # Top 10 estados por preço
    top_states = state_avg.head(10)
    
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    
    # Gráfico de barras
    bars = axes[0].bar(range(len(top_states)), top_states['mean'], 
                       yerr=top_states['std'], capsize=5, color='steelblue', alpha=0.8)
    axes[0].set_title('🏆 Top 10 Estados - Preço Médio', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Preço Médio (R$/arroba)', fontsize=12)
    axes[0].set_xticks(range(len(top_states)))
    axes[0].set_xticklabels(top_states.index, rotation=45, ha='right')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height + top_states['std'].iloc[i],
                    f'R$ {height:.1f}', ha='center', va='bottom', fontsize=9)
    
    # Boxplot por estado (top 8)
    top_8_states = top_states.head(8).index
    data_for_box = [ts_data[ts_data['state'] == state]['price_brl'] for state in top_8_states]
    
    box_plot = axes[1].boxplot(data_for_box, labels=[s[:10] for s in top_8_states], patch_artist=True)
    axes[1].set_title('📦 Distribuição de Preços - Top 8 Estados', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Colorir boxplots
    colors = plt.cm.Set3(np.linspace(0, 1, len(box_plot['boxes'])))
    for patch, color in zip(box_plot['boxes'], colors):
        patch.set_facecolor(color)
    
    plt.tight_layout()
    plt.show()
    
    # Tabela resumo
    print("📊 RESUMO POR ESTADO (Top 10):")
    print("=" * 80)
    print(f"{'Estado':<20} {'Preço Médio':<12} {'Desvio':<10} {'Registros':<10}")
    print("=" * 80)
    for state, row in top_states.iterrows():
        print(f"{state:<20} R$ {row['mean']:<8.2f} ±{row['std']:<7.2f} {row['count']:<10.0f}")

## 6. Análise de Correlações

In [ ]:
if not ts_data.empty:
    # Criar matriz de correlação entre estados
    pivot_states = loader.create_pivot_table(
        ts_data, 
        value_col='price_brl', 
        index_col='date', 
        columns_col='state'
    )
    
    if not pivot_states.empty and pivot_states.shape[1] > 1:
        # Calcular correlação
        correlation_matrix = pivot_states.corr()
        
        # Gráfico de correlação
        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Máscara para triângulo superior
        
        sns.heatmap(correlation_matrix, 
                   mask=mask,
                   annot=True, 
                   cmap='RdYlBu_r', 
                   center=0,
                   square=True,
                   fmt='.2f',
                   cbar_kws={"shrink": .8})
        
        plt.title('🔗 Matriz de Correlação - Preços entre Estados', fontsize=16, fontweight='bold', pad=20)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
        
        # Encontrar correlações mais altas e baixas
        correlation_matrix = correlation_matrix.where(~mask)
        correlation_flat = correlation_matrix.stack().dropna()
        
        print("🔗 CORRELAÇÕES MAIS ALTAS:")
        print("=" * 50)
        top_corr = correlation_flat.nlargest(5)
        for (state1, state2), corr in top_corr.items():
            print(f"{state1} ↔ {state2}: {corr:.3f}")
        
        print("\n🔗 CORRELAÇÕES MAIS BAIXAS:")
        print("=" * 50)
        bottom_corr = correlation_flat.nsmallest(5)
        for (state1, state2), corr in bottom_corr.items():
            print(f"{state1} ↔ {state2}: {corr:.3f}")

## 7. Análise Sazonal

In [ ]:
if not ts_data.empty:
    # Adicionar features temporais
    ts_analysis = ts_data.reset_index()
    ts_analysis['month'] = ts_analysis['date'].dt.month
    ts_analysis['quarter'] = ts_analysis['date'].dt.quarter
    ts_analysis['day_of_week'] = ts_analysis['date'].dt.dayofweek
    ts_analysis['month_name'] = ts_analysis['date'].dt.month_name()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Análise por mês
    monthly_data = ts_analysis.groupby('month')['price_brl'].agg(['mean', 'std'])
    month_names = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
                  'Jul', 'Aug', 'Set', 'Out', 'Nov', 'Dez']
    
    axes[0,0].bar(monthly_data.index, monthly_data['mean'], 
                  yerr=monthly_data['std'], capsize=5, color='lightcoral', alpha=0.8)
    axes[0,0].set_title('📅 Preço Médio por Mês', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[0,0].set_xticks(monthly_data.index)
    axes[0,0].set_xticklabels([month_names[i-1] for i in monthly_data.index])
    axes[0,0].grid(True, alpha=0.3, axis='y')
    
    # Análise por trimestre
    quarterly_data = ts_analysis.groupby('quarter')['price_brl'].mean()
    axes[0,1].bar(quarterly_data.index, quarterly_data.values, color='lightgreen', alpha=0.8)
    axes[0,1].set_title('📊 Preço Médio por Trimestre', fontsize=14, fontweight='bold')
    axes[0,1].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[0,1].set_xlabel('Trimestre')
    axes[0,1].set_xticks(quarterly_data.index)
    axes[0,1].grid(True, alpha=0.3, axis='y')
    
    # Análise por dia da semana
    daily_data = ts_analysis.groupby('day_of_week')['price_brl'].mean()
    day_names = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom']
    
    axes[1,0].bar(daily_data.index, daily_data.values, color='lightblue', alpha=0.8)
    axes[1,0].set_title('📆 Preço Médio por Dia da Semana', fontsize=14, fontweight='bold')
    axes[1,0].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[1,0].set_xticks(daily_data.index)
    axes[1,0].set_xticklabels(day_names)
    axes[1,0].grid(True, alpha=0.3, axis='y')
    
    # Boxplot mensal
    sns.boxplot(data=ts_analysis, x='month', y='price_brl', ax=axes[1,1])
    axes[1,1].set_title('📦 Distribuição Mensal dos Preços', fontsize=14, fontweight='bold')
    axes[1,1].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[1,1].set_xlabel('Mês')
    axes[1,1].set_xticklabels([month_names[i-1] for i in range(1, 13)])
    axes[1,1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas sazonais
    print("📅 ANÁLISE SAZONAL:")
    print("=" * 50)
    
    highest_month = monthly_data['mean'].idxmax()
    lowest_month = monthly_data['mean'].idxmin()
    
    print(f"🔝 Mês com maior preço: {month_names[highest_month-1]} (R$ {monthly_data['mean'][highest_month]:.2f})")
    print(f"🔻 Mês com menor preço: {month_names[lowest_month-1]} (R$ {monthly_data['mean'][lowest_month]:.2f})")
    print(f"📊 Diferença sazonal: R$ {monthly_data['mean'][highest_month] - monthly_data['mean'][lowest_month]:.2f}")
    
    highest_quarter = quarterly_data.idxmax()
    lowest_quarter = quarterly_data.idxmin()
    print(f"\n🔝 Melhor trimestre: Q{highest_quarter} (R$ {quarterly_data[highest_quarter]:.2f})")
    print(f"🔻 Pior trimestre: Q{lowest_quarter} (R$ {quarterly_data[lowest_quarter]:.2f})")

## 8. Análise de Outliers

In [ ]:
if not ts_data.empty:
    # Detectar outliers usando IQR
    Q1 = ts_data['price_brl'].quantile(0.25)
    Q3 = ts_data['price_brl'].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = ts_data[(ts_data['price_brl'] < lower_bound) | (ts_data['price_brl'] > upper_bound)]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Histograma com outliers marcados
    axes[0].hist(ts_data['price_brl'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].axvline(lower_bound, color='red', linestyle='--', alpha=0.8, label=f'Limite Inferior: R$ {lower_bound:.2f}')
    axes[0].axvline(upper_bound, color='red', linestyle='--', alpha=0.8, label=f'Limite Superior: R$ {upper_bound:.2f}')
    axes[0].axvline(ts_data['price_brl'].mean(), color='green', linestyle='-', alpha=0.8, label=f'Média: R$ {ts_data["price_brl"].mean():.2f}')
    
    axes[0].set_title('📊 Distribuição de Preços com Outliers', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Preço (R$/arroba)', fontsize=12)
    axes[0].set_ylabel('Frequência', fontsize=12)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Série temporal com outliers destacados
    daily_avg = ts_data.groupby('date')['price_brl'].mean()
    outliers_daily = outliers.groupby('date')['price_brl'].mean()
    
    axes[1].plot(daily_avg.index, daily_avg.values, linewidth=2, color='blue', alpha=0.7, label='Preços Normais')
    
    if not outliers_daily.empty:
        axes[1].scatter(outliers_daily.index, outliers_daily.values, 
                       color='red', s=50, alpha=0.8, label=f'Outliers ({len(outliers_daily)})', zorder=5)
    
    axes[1].set_title('🎯 Outliers na Série Temporal', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Data', fontsize=12)
    axes[1].set_ylabel('Preço (R$/arroba)', fontsize=12)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Relatório de outliers
    print("🎯 ANÁLISE DE OUTLIERS:")
    print("=" * 50)
    print(f"Total de registros: {len(ts_data)}")
    print(f"Outliers encontrados: {len(outliers)} ({len(outliers)/len(ts_data)*100:.1f}%)")
    print(f"Limite inferior: R$ {lower_bound:.2f}")
    print(f"Limite superior: R$ {upper_bound:.2f}")
    
    if not outliers.empty:
        print("\n🔍 OUTLIERS EXTREMOS:")
        extreme_outliers = outliers.nlargest(5, 'price_brl')
        for idx, row in extreme_outliers.iterrows():
            print(f"  {row['state']} - {idx.strftime('%d/%m/%Y')}: R$ {row['price_brl']:.2f}")

## 9. Resumo da Análise

In [ ]:
if not ts_data.empty:
    print("📋 RESUMO EXECUTIVO DA ANÁLISE")
    print("=" * 70)
    print(f"📅 Período analisado: {ts_data.index.min().strftime('%d/%m/%Y')} até {ts_data.index.max().strftime('%d/%m/%Y')}")
    print(f"📊 Total de registros: {len(ts_data):,}")
    print(f"🗺️ Estados analisados: {ts_data['state'].nunique()}")
    print(f"📈 Indicadores analisados: {ts_data['indicator_name'].nunique()}")
    print()
    
    print("💰 PREÇOS:")
    print(f"   Preço médio geral: R$ {ts_data['price_brl'].mean():.2f}")
    print(f"   Menor preço: R$ {ts_data['price_brl'].min():.2f}")
    print(f"   Maior preço: R$ {ts_data['price_brl'].max():.2f}")
    print(f"   Desvio padrão: R$ {ts_data['price_brl'].std():.2f}")
    print()
    
    # Estados com maiores e menores preços
    state_avg = ts_data.groupby('state')['price_brl'].mean().sort_values(ascending=False)
    print("🏆 RANKINGS:")
    print(f"   Estado com maior preço: {state_avg.index[0]} (R$ {state_avg.iloc[0]:.2f})")
    print(f"   Estado com menor preço: {state_avg.index[-1]} (R$ {state_avg.iloc[-1]:.2f})")
    print(f"   Diferença entre extremos: R$ {state_avg.iloc[0] - state_avg.iloc[-1]:.2f}")
    print()
    
    # Variação temporal
    daily_avg = ts_data.groupby('date')['price_brl'].mean()
    first_price = daily_avg.iloc[0]
    last_price = daily_avg.iloc[-1]
    total_variation = ((last_price - first_price) / first_price) * 100
    
    print("📈 VARIAÇÃO TEMPORAL:")
    print(f"   Preço inicial: R$ {first_price:.2f}")
    print(f"   Preço final: R$ {last_price:.2f}")
    print(f"   Variação total: {total_variation:+.2f}%")
    
    # Volatilidade
    daily_returns = daily_avg.pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(252) * 100  # Anualizada
    print(f"   Volatilidade anualizada: {volatility:.1f}%")
    print()
    
    print("🎯 PRÓXIMOS PASSOS RECOMENDADOS:")
    print("   1. Feature Engineering para modelos ML")
    print("   2. Análise de séries temporais (ARIMA, Prophet)")
    print("   3. Modelos de previsão de preços")
    print("   4. Análise de arbitragem entre estados")
    print("   5. Sistema de alertas para anomalias")
    
else:
    print("❌ Nenhum dado disponível para análise.")
    print("💡 Certifique-se de que:")
    print("   1. O ETL foi executado com sucesso")
    print("   2. Os dados estão no caminho correto")
    print("   3. Existem dados para o período especificado")

## 10. Exportar Resultados

In [ ]:
if not ts_data.empty:
    # Criar resumos para exportação
    os.makedirs('../reports', exist_ok=True)
    
    # Resumo por estado
    state_summary = ts_data.groupby('state').agg({
        'price_brl': ['count', 'mean', 'std', 'min', 'max'],
        'variation_pct': ['mean', 'std']
    }).round(2)
    
    state_summary.columns = ['_'.join(col).strip() for col in state_summary.columns]
    state_summary.to_csv('../reports/resumo_por_estado.csv', encoding='utf-8')
    print("✅ Resumo por estado salvo em: ../reports/resumo_por_estado.csv")
    
    # Série temporal diária
    daily_summary = ts_data.groupby('date').agg({
        'price_brl': ['mean', 'std', 'min', 'max', 'count']
    }).round(2)
    
    daily_summary.columns = ['_'.join(col).strip() for col in daily_summary.columns]
    daily_summary.to_csv('../reports/serie_temporal_diaria.csv', encoding='utf-8')
    print("✅ Série temporal diária salva em: ../reports/serie_temporal_diaria.csv")
    
    # Dataset completo para análises futuras
    ts_data.reset_index().to_csv('../reports/dados_completos.csv', index=False, encoding='utf-8')
    print("✅ Dados completos salvos em: ../reports/dados_completos.csv")
    
    print("\n🎉 Análise exploratória concluída com sucesso!")
    print("📂 Todos os resultados foram salvos na pasta 'reports'")
    
else:
    print("❌ Não foi possível exportar os resultados - dados não disponíveis.")